<font size="8"><center>Trading Cards on eBay - Data Cleaning and Preprocessing
</center></font>



<div class="alert alert-warning">

__Author__: -  Kasun Malwenna
    

__Contact__: - kasunmalwenna@gmail.com

---

This Notebook's objective is to gather Sports Trading card listings from eBay and execute data cleaning and wragling processes in order to prepare the data for exploratory data analysis. The analysis on the data is documented in *__eBay_data_analysis.ipynb__* notebook.

---
</div>

># Data Collection

Current PSA graded Sports Trading Card listings on eBay USA were collected using [Apify eBay scrapper](https://apify.com/dtrungtin/ebay-items-scraper). Apify is a web scraping and automation platform that serves as an unofficial eBay API to extract data from eBay. It allows us to specify the product category, keywords, and other parameters to search for, and then it will extract data such as product name, price, description, and image URLs from the search results. Data was collected on 10th December 2022 and represents the most up-to-date "Buy now" listings on eBay and was exported to multiple CSV files.

### Loading the Data

In [1]:
# Standard library imports
import pandas as pd
import numpy as np

# Library to search for a specific file patterns
import glob

First, let's read in data from multiple CSV files saved in our local directory. We will use __glob__ library to search for a specific file pattern.

In [2]:
# Define the path to the data and pattern to search for
file_paths = glob.glob('psa data/*.csv')

# Read the data into a single DataFrame
df = pd.concat((pd.read_csv(file) for file in file_paths), ignore_index=True)


In [4]:
#  Print the shape of the DataFrame
df.shape

print(f'Dataframe has {df.shape[0]} rows and  {df.shape[1]} columns')

Dataframe has 104787 rows and  32 columns


Let's inspect the first 5 rows of the DataFrame.

In [5]:
# display all columns in dataframe
pd.set_option('display.max_columns', None)

# show the first 5 rows
df.head()

,available,availableText,brand,ean,endedDate,itemLocation,itemNumber,mpn,price,priceWithCurrency,seller,sold,subTitle,title,type,upc,url,wasPrice,wasPriceWithCurrency,whyToBuy/0,whyToBuy/1,whyToBuy/2,image,whyToBuy/3,whyToBuy/4,whyToBuy/5,item_country,item_state,item_city,card_year,card_brand,card_singed
0,NaN,NaN,NaN,NaN,NaN,"Dearborn, Michigan, United States",3.242724e+11,NaN,12630069.0,"US $12,630,069.00",nfarj,NaN,Best Ever! The Best PSA Serial #’d Card on the...,1986 86 Fleer Michael Jordan Rookie Sticker PS...,Sports Trading Card,NaN,https://www.ebay.com/itm/324272400062,NaN,NaN,Ships from United States,349 watchers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,"Livonia, Michigan, United States",1.950833e+11,NaN,7000000.0,"US $7,000,000.00",cardguy444,NaN,HOLY GRAIL RAREST S/N TOM BRADY PSA 10 RC IN T...,SUPER RARE ( ELITE STATUS ) 2000 TOM BRADY ROO...,Sports Trading Card,NaN,https://www.ebay.com/itm/195083288951,NaN,NaN,Ships from United States,192 watchers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,"San Antonio, Texas, United States",2.850249e+11,NaN,5000000.0,"US $5,000,000.00",sa-techboy,NaN,NaN,2003 LEBRON JAMES RC UD ROOKIE EXCLUSIVES AUTO...,Sports Trading Card,NaN,https://www.ebay.com/itm/285024859926,NaN,NaN,Ships from United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,"Livonia, Michigan, United States",1.955095e+11,NaN,4500000.0,"US $4,500,000.00",cardguy444,NaN,NaN,2003 TOM BRADY LEAF LIMITED # 12/12 GAME-WORN-...,Sports Trading Card,NaN,https://www.ebay.com/itm/195509543378,NaN,NaN,Ships from United States,8 watchers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,"Indianapolis, Indiana, United States",2.040748e+11,NaN,3900000.0,"US $3,900,000.00",tb12mb07,NaN,NaN,RARE 2007 Tom Brady Autograph Plaque 16-0 #Und...,Sports Trading Card,NaN,https://www.ebay.com/itm/204074772841,NaN,NaN,Ships from United States,9 watchers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We should inspect the dataset to highlight if any cleaning is required.

In [6]:
# Dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104787 entries, 0 to 104786
Data columns (total 32 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   available             1157 non-null    float64
 1   availableText         1157 non-null    object 
 2   brand                 287 non-null     object 
 3   ean                   16 non-null      object 
 4   endedDate             1 non-null       object 
 5   itemLocation          60447 non-null   object 
 6   itemNumber            60473 non-null   float64
 7   mpn                   72 non-null      object 
 8   price                 104768 non-null  float64
 9   priceWithCurrency     60454 non-null   object 
 10  seller                59996 non-null   object 
 11  sold                  146 non-null     float64
 12  subTitle              2094 non-null    object 
 13  title                 104767 non-null  object 
 14  type                  53995 non-null   object 
 15  

It seems there are lot of columns with missing values that needs to be dealt with.

---
># Data Cleaning

### Check for Missing Values

Let's look at the missing values for each column.

In [7]:
# Check  missing values
df.isnull().sum()

available               103630
availableText           103630
brand                   104500
ean                     104771
endedDate               104786
itemLocation             44340
itemNumber               44314
mpn                     104715
price                       19
priceWithCurrency        44333
seller                   44791
sold                    104641
subTitle                102693
title                       20
type                     50792
upc                      99690
url                          0
wasPrice                104787
wasPriceWithCurrency    104787
whyToBuy/0                3969
whyToBuy/1               79299
whyToBuy/2              101843
image                   104786
whyToBuy/3              104745
whyToBuy/4              104785
whyToBuy/5              104785
item_country             60473
item_state               60473
item_city                60473
card_year                60473
card_brand               60473
card_singed              60473
dtype: i

To get a better understating of the proportion, let's check missing values as a percentage of the total number of values in each column.

In [8]:
# check missing value precentage
df.isnull().sum()/len(df)*100


available                98.895855
availableText            98.895855
brand                    99.726111
ean                      99.984731
endedDate                99.999046
itemLocation             42.314409
itemNumber               42.289597
mpn                      99.931289
price                     0.018132
priceWithCurrency        42.307729
seller                   42.744806
sold                     99.860670
subTitle                 98.001661
title                     0.019086
type                     48.471662
upc                      95.135847
url                       0.000000
wasPrice                100.000000
wasPriceWithCurrency    100.000000
whyToBuy/0                3.787684
whyToBuy/1               75.676372
whyToBuy/2               97.190491
image                    99.999046
whyToBuy/3               99.959919
whyToBuy/4               99.998091
whyToBuy/5               99.998091
item_country             57.710403
item_state               57.710403
item_city           

We observe a number of columns with a large proportion of missing values. Some are even missing more than 90% of the data. As these columns will be redundant for our analysis, we can drop them.
To begin with, 50% of missing data in a column can be a good threshold to remove the column entirely.


In [9]:
# drop coulumns with more than 50% missing values
df.dropna(thresh=len(df)*0.5, axis=1, inplace=True)
#check
df.head()

,itemLocation,itemNumber,price,priceWithCurrency,seller,title,type,url,whyToBuy/0
0,"Dearborn, Michigan, United States",3.242724e+11,12630069.0,"US $12,630,069.00",nfarj,1986 86 Fleer Michael Jordan Rookie Sticker PS...,Sports Trading Card,https://www.ebay.com/itm/324272400062,Ships from United States
1,"Livonia, Michigan, United States",1.950833e+11,7000000.0,"US $7,000,000.00",cardguy444,SUPER RARE ( ELITE STATUS ) 2000 TOM BRADY ROO...,Sports Trading Card,https://www.ebay.com/itm/195083288951,Ships from United States
2,"San Antonio, Texas, United States",2.850249e+11,5000000.0,"US $5,000,000.00",sa-techboy,2003 LEBRON JAMES RC UD ROOKIE EXCLUSIVES AUTO...,Sports Trading Card,https://www.ebay.com/itm/285024859926,Ships from United States
3,"Livonia, Michigan, United States",1.955095e+11,4500000.0,"US $4,500,000.00",cardguy444,2003 TOM BRADY LEAF LIMITED # 12/12 GAME-WORN-...,Sports Trading Card,https://www.ebay.com/itm/195509543378,Ships from United States
4,"Indianapolis, Indiana, United States",2.040748e+11,3900000.0,"US $3,900,000.00",tb12mb07,RARE 2007 Tom Brady Autograph Plaque 16-0 #Und...,Sports Trading Card,https://www.ebay.com/itm/204074772841,Ships from United States


In [11]:
# check
df.isnull().sum()/len(df)*100

itemLocation         42.314409
itemNumber           42.289597
price                 0.018132
priceWithCurrency    42.307729
seller               42.744806
title                 0.019086
type                 48.471662
url                   0.000000
whyToBuy/0            3.787684
dtype: float64

We now have only 9 columns left where some columns still have a large number of missing values. Let's check columns one by one and deal with them accordingly.

### Cleaning by Each Column

#### `itemNumber` Column

In [12]:
# Inspect the first 5 rows of the itemNumber column
df.itemNumber.head()

0    3.242724e+11
1    1.950833e+11
2    2.850249e+11
3    1.955095e+11
4    2.040748e+11
Name: itemNumber, dtype: float64

These are just numbers assigned to each item that does not relate to any meaningful information. So we will drop it.

In [13]:
# drop column 
df.drop(['itemNumber'], axis=1, inplace=True)


#### `type` column

In [14]:
# Inspect unique values in the type column
df.type.unique()

array(['Sports Trading Card', nan, 'Sports Trading Cards', 'RARE', 'none',
       'Non-sports Trading Card', 'Basketball-NBA/Unsigned Memoribilia',
       'DDA', 'Hockey-NHL/Unsigned Memorabilia', 'Single',
       'Sports Trading Card2018 Panini', 'mlb singles',
       'Non-Sport Trading Card', 'Action Figure',
       'Trading Cards/ Stickers', 'Autograph Ball', 'Topps', 'Rookie',
       'Trading Card', 'nba singles', 'None', 'Sports'], dtype=object)

Information from type column is not useful as it is unclear and doesn't tell us about the type of sport.so we will drop it.`

In [15]:
# drop column
df.drop(['type'], axis=1, inplace=True)

# check
df.columns

Index(['itemLocation', 'price', 'priceWithCurrency', 'seller', 'title', 'url',
       'whyToBuy/0'],
      dtype='object')

#### `priceWithCurrency` column

In [18]:
# Inspect unique values 
df.priceWithCurrency.unique()

array(['US $12,630,069.00', 'US $7,000,000.00', 'US $5,000,000.00', ...,
       'C $450.00', 'US $424.00', 'US $237.83'], dtype=object)

the column seems to have  same information as the column 'price'. Let's double check this claim. 

In [19]:
# check
df[['price', 'priceWithCurrency']].head()

,price,priceWithCurrency
0,12630069.0,"US $12,630,069.00"
1,7000000.0,"US $7,000,000.00"
2,5000000.0,"US $5,000,000.00"
3,4500000.0,"US $4,500,000.00"
4,3900000.0,"US $3,900,000.00"


We can drop the column `priceWithCurrency` as it is duplicate of `price` column.

In [20]:
# drop column
df.drop(['priceWithCurrency'], axis=1, inplace=True)

# check
df.columns

Index(['itemLocation', 'price', 'seller', 'title', 'url', 'whyToBuy/0'], dtype='object')

#### `whyToBuy/0` colum


In [21]:
# Inspect unique value counts
df['whyToBuy/0'].value_counts()

Ships from United States     77743
Returns accepted             10698
Free shipping and returns     3249
Unknown                       2818
5 watchers                     458
                             ...  
127 watchers                     1
96 watchers                      1
92 watchers                      1
183 watchers                     1
58 watchers                      1
Name: whyToBuy/0, Length: 167, dtype: int64

This information can be usefull for the analysis. Less than 4% of data is missing from the column. Therefore, we can impute the missing values with 'Unknwon' string.

In [22]:
# Impute missing values with a string 'Unknown'
df['whyToBuy/0'].fillna('Unknown', inplace=True)

# check
df['whyToBuy/0'].isnull().sum()


0

#### `seller` column

In [23]:
# Inspect unique values
df.seller.unique()

array(['nfarj', 'cardguy444', 'sa-techboy', ..., 'kennynell',
       'bigglesworth00', 'timboslice38'], dtype=object)

Not a useful column. 

In [24]:
# drop seller column
df.drop(['seller'], axis=1, inplace=True)

# check
df.columns

Index(['itemLocation', 'price', 'title', 'url', 'whyToBuy/0'], dtype='object')

#### `itemLoaction` column

In [25]:
# Inspect unique values
df.itemLocation.unique()

array(['Dearborn, Michigan, United States',
       'Livonia, Michigan, United States',
       'San Antonio, Texas, United States', ..., 'Fredericton, Canada',
       'Baden, Canada', 'Ardmore, Alabama, United States'], dtype=object)

We can do feature extraction from `itemLocation` column. But first, we need to deal with the missing values. There is no real way to impute missing values as we do not have any other columns pointing towards geographical information on the listing.
The next option is to drop only the rows with missing values which take up nearly 43% of data from the total data. This will severely reduce the number of observations we have and is not the best approach. However, since we want to perform a market analysis on trading cards, and location information for that is crucial, let’s go ahead and drop only the missing observations.


In [26]:
# Current shape of the DataFrame
df.shape

(104787, 5)

In [27]:
# drop missing rows in itemLocation column
df.dropna(subset=['itemLocation'], inplace=True)

# check
df.isnull().sum()

itemLocation    0
price           0
title           0
url             0
whyToBuy/0      0
dtype: int64

In [28]:
# New shape of the DataFrame
df.shape

(60447, 5)

We have dealt with all the missing values in our data. However, in doing so, we reduced our observations from 104,787 to 60,447 and features from 32 to 5.

---
># Data Preprocessing

### Feature Extraction from `itemLocation` Column

We can extract country, state and city from the itemLocation column.

In [29]:
# Extract the country name from the itemLocation column
df['item_country'] = df['itemLocation'].str.split(',').str[-1]

# Extracy the state name from the itemLocation column
df['item_state'] = df['itemLocation'].str.split(',').str[-2]

# Extract the city name from the itemLocation column
df['item_city'] = df['itemLocation'].str.split(',').str[0]

# Drop the itemLocation column
df.drop(['itemLocation'], axis=1, inplace=True)

# check
df.columns

Index(['price', 'title', 'url', 'whyToBuy/0', 'item_country', 'item_state',
       'item_city'],
      dtype='object')

In [30]:
# Check
df.head()

,price,title,url,whyToBuy/0,item_country,item_state,item_city
0,12630069.0,1986 86 Fleer Michael Jordan Rookie Sticker PS...,https://www.ebay.com/itm/324272400062,Ships from United States,United States,Michigan,Dearborn
1,7000000.0,SUPER RARE ( ELITE STATUS ) 2000 TOM BRADY ROO...,https://www.ebay.com/itm/195083288951,Ships from United States,United States,Michigan,Livonia
2,5000000.0,2003 LEBRON JAMES RC UD ROOKIE EXCLUSIVES AUTO...,https://www.ebay.com/itm/285024859926,Ships from United States,United States,Texas,San Antonio
3,4500000.0,2003 TOM BRADY LEAF LIMITED # 12/12 GAME-WORN-...,https://www.ebay.com/itm/195509543378,Ships from United States,United States,Michigan,Livonia
4,3900000.0,RARE 2007 Tom Brady Autograph Plaque 16-0 #Und...,https://www.ebay.com/itm/204074772841,Ships from United States,United States,Indiana,Indianapolis


### Feature Extraction from `title` Column

In [31]:
# check unique values in title column
df['title'].unique()

array(['1986 86 Fleer Michael Jordan Rookie Sticker PSA 8(ST) # 06300690 CAREER HIGH PTS',
       'SUPER RARE ( ELITE STATUS ) 2000 TOM BRADY ROOKIE DONURSS S/N 05/10 PSA 10 POP 1',
       '2003 LEBRON JAMES RC UD ROOKIE EXCLUSIVES AUTO GEM MINT Rare Highest Graded',
       ..., '1955 BOWMAN MICKEY MANTLE #202 PSA 1',
       '2019-20 Panini Mosaic Lebron James Silver Mosaic Prizm PSA Gem Mint 10 #8 Lakers',
       '2015-16 PANINI PRIZM DEVIN BOOKER PSA 10 ROOKIE RC #308 PHOENIX SUNS'],
      dtype=object)

##### Extracting card year

Year is a 4 digit number that starts from 19 or 20. We can ectract such numbers and save it to a new column.

In [58]:
# Grab year from title column that start with '20' and '19' and has 4 digits
df['card_year'] = df['title'].str.extract(r'((?:19|20)\d\d)')

In [139]:
# Grab year from title column
df['card_year'] = df['title'].str.extract('(\d{4})', expand=True)

In [59]:
# Inspect the first 10 values of the card_year column
df.card_year.head(10)

0    1986
1    2000
2    2003
3    2003
4    2007
5    1947
6    2015
7    2003
8    2000
9    2003
Name: card_year, dtype: object

In [60]:
# Check for missing years in new feature
df.card_year.isnull().sum()

2455

There are some missing values in the new column. Let’s inspect them closely.

In [61]:
# Inspect Nan values from card_year column
df[df['card_year'].isnull()][['card_year','url','title']].head(5)

,card_year,url,title
45,NaN,https://www.ebay.com/itm/324165986164,PSA TRUE 1/1! DUAL AUTO CRAZY RAY & CHIEF ZEE ...
55,NaN,https://www.ebay.com/itm/225148828552,09-10 Playoff Contenders Championship & Gold T...
63,NaN,https://www.ebay.com/itm/334009075378,Khabib Nurmagomedov Auto /25 Panini UFC Champi...
82,NaN,https://www.ebay.com/itm/144254733247,1/1 MICHAEL JORDAN Complete 22KT GOLD SET BGS 9
98,NaN,https://www.ebay.com/itm/275524977554,BGS 10 PRISTINE BLACK LABEL SP AUTHENTIC NATHA...


Seems like most year values that are missing from the title column are card sets, not individual cards. Since we are only exploring individual cards in the analysis, We can drop these rows.

In [62]:
# drop rows with Nan values in card_year column
df.dropna(subset=['card_year'], inplace=True)


In [63]:
# Re check
df.card_year.isnull().sum()

0

In [64]:
# Inspect unique values in card_year column
df.card_year.unique()

array(['1986', '2000', '2003', '2007', '1947', '2015', '2018', '2020',
       '2009', '1985', '1997', '2019', '2004', '1984', '1993', '2016',
       '1996', '2002', '1968', '1998', '1921', '2005', '2006', '2014',
       '2013', '2011', '2001', '1992', '1911', '1952', '1999', '2017',
       '2012', '1995', '2021', '1932', '1916', '1990', '2008', '1910',
       '1902', '1933', '1937', '1914', '2010', '1909', '1948', '1979',
       '1936', '1989', '1969', '1934', '1959', '1954', '1951', '1980',
       '1970', '1965', '1987', '1925', '2022', '1981', '1956', '1963',
       '1973', '1976', '1975', '1966', '1957', '1958', '1955', '1991',
       '1912', '1962', '1961', '1945', '1923', '1913', '1924', '1988',
       '1994', '1971', '1960', '1949', '1939', '1922', '1953', '1915',
       '1917', '1926', '1974', '1935', '1964', '1950', '1967', '1983',
       '1977', '1982', '1938', '1907', '1978', '1928', '1940', '1946',
       '1972', '1927', '1920', '1931', '1929', '1903', '1919', '1918',
      

#### Extracting brand name 

In [65]:
# show entire value in title column
pd.set_option('display.max_colwidth', None)

In [66]:
# Inspect the column to identify patterns
df.title.head(57)

0     1986 86 Fleer Michael Jordan Rookie Sticker PSA 8(ST) # 06300690 CAREER HIGH PTS
1     SUPER RARE ( ELITE STATUS ) 2000 TOM BRADY ROOKIE DONURSS S/N 05/10 PSA 10 POP 1
2          2003 LEBRON JAMES RC UD ROOKIE EXCLUSIVES AUTO GEM MINT Rare Highest Graded
3     2003 TOM BRADY LEAF LIMITED # 12/12 GAME-WORN-2002-PRO-BOWL-JERSEY-AUTO-BGS 9/10
4     RARE 2007 Tom Brady Autograph Plaque 16-0 #Undefeated #GreatnessCode #RookieCard
5        1947 Jackie Robinson Rookie RC Brooklyn Dodgers 1st MLB Appearance PSA 9 Mint
6       2015-16 Exquisite Connor McDavid 1/1 RPA NHL Shield Logo Patch BGS 9.5 10 AUTO
7      Lebron James 2003 SP Signature Edition RC Inscription 'The KING' Auto /25 BGS10
8     RARE ( PLAYERS HONORS )(#/10) 2000 TOM BRADY ROOKIE ABSOLUTE PLAYOFF PSA 9 POP 1
9     2003-04 UD Rookie Exclusives AUTOGRAPH Michael Jordan A60 BGS PRISTINE 10 POP 1!
10         2018-19 PANINI CONTENDERS LUKA DONCIC PREMIUM GOLD RC AUTO BGS 9.5/10 POP 1
11    2020 Topps Chrome F1 Lewis Hamilton S

In order to extract the brand, we need to check the column values against a list of brand names. We can manually create a list of brands by exploring the brand filter option on eBay, where we can filter the listing by brand name.
The following are brand names obtained in such a way, saved to a string we can convert to a list. 


In [67]:
# save most popular brands as a string 
string = "5FINITY,7th Inning Sketch,AAA Sports Memorabilia,Ace Authentic,Action Packed,AEG,Allens,American Caramel,APBA,ArtBox,Atlantic,AW Sports, Inc.,BandaiBaseball Magazine,Bazooka, Be A Player, Bench Warmer, Bleachers, Bowman, Bushiroad, Calbee, Callahan HALL OF FAME, CARDZ Distribution, Champion Cards, Choice Marketing, Inc, Chris Martin Enterprises, Inc., Classic Games, Inc, Classic Marketing, Inc., CMC, Coles, Collector's Edge, College Classics, Inc., Collegiate Collection, Comic Images, Courtside, Cracker Jack, Cryptozoic, Dan Dee, Diamond Kings, Diamond Stars, Disney, Donruss, Donruss, Playoff, Double Play, Eclipse Cards, Enor, Exhibits, Fantasy Flight Games, Félix Potin, Fleer, Fleer/SkyBox International, Futera, Goal Line, Goal Line Art, Goudey, Grandstand, Hasbro, Hi-Pro Marketing, Inc., Homogenized Bond, Impel, Imperial Tobacco Canada, In the Game, JOGO, Inc., Just Memorabilia, Just Minors, Kahn's, Kellogg's, Kenner, Konami, Leaf, Marvel, Merlin, Milk Duds, Mobil, Monty Gum, MSA, National Chicle, Navarrete, NBA Properties, NetPro, NFL Players Association, NFL Properties, Nintendo, NSI Marketing Limited, Nu-Cards, Onyx Authenticated, O-Pee-Chee, Pacific, Panini, Parkhurst, Parkside, Philadelphia Gum, Pinnacle, Playoff, Post, Press Pass, ProCards, Pro Set, R.E.L., Razor, Red Man, Regina, Ringside, Rittenhouse, Roox Limited Corp., SAGE, Scanlens, SCORE, Select, Sereal, Shirriff Coins, Signature Rookies, SkyBox, Sp Authentic, Sportflics, Sport Kings, Sportscaster, Sports Illustrated, SPX, Stadium Club, Star Pics, Superior Pix, Superior Rookies, TCMA, Team Issue, The Score Board, Inc, The Star Co., Topps, Topps Supreme, Tristar, Ty, Ultimate Guard, Ultimate Trading Card Co., Ultra PRO, Upper Deck, Vachon, Wheaties, Wheels, Wild Card, Wills, Wizards of the Coast, WOW, Unbranded,Not Specified,5FINITY,7TH INNING SKETCH,AAA SPORTS MEMORABILIA,ACE AUTHENTIC,ACTION PACKED,AEG,ALLENS,AMERICAN CARAMEL,APBA,ARTBOX,ATLANTIC,AW SPORTS, INC.,BANDAIBASEBALL MAGAZINE,BAZOOKA, BE A PLAYER, BENCH WARMER, BLEACHERS, BOWMAN, BUSHIROAD, CALBEE, CALLAHAN HALL OF FAME, CARDZ DISTRIBUTION, CHAMPION CARDS, CHOICE MARKETING, INC, CHRIS MARTIN ENTERPRISES, INC., CLASSIC GAMES, INC, CLASSIC MARKETING, INC., CMC, COLES, COLLECTOR'S EDGE, COLLEGE CLASSICS, INC., COLLEGIATE COLLECTION, COMIC IMAGES, COURTSIDE, CRACKER JACK, CRYPTOZOIC, DAN DEE, DIAMOND KINGS, DIAMOND STARS, DISNEY, DONRUSS, DONRUSS, PLAYOFF, DOUBLE PLAY, ECLIPSE CARDS, ENOR, EXHIBITS, FANTASY FLIGHT GAMES, FÉLIX POTIN, FLEER, FLEER/SKYBOX INTERNATIONAL, FUTERA, GOAL LINE, GOAL LINE ART, GOUDEY, GRANDSTAND, HASBRO, HI-PRO MARKETING, INC., HOMOGENIZED BOND, IMPEL, IMPERIAL TOBACCO CANADA, IN THE GAME, JOGO, INC., JUST MEMORABILIA, JUST MINORS, KAHN'S, KELLOGG'S, KENNER, KONAMI, LEAF, MARVEL, MERLIN, MILK DUDS, MOBIL, MONTY GUM, MSA, NATIONAL CHICLE, NAVARRETE, NBA PROPERTIES, NETPRO, NFL PLAYERS ASSOCIATION, NFL PROPERTIES, NINTENDO, NSI MARKETING LIMITED, NU-CARDS, ONYX AUTHENTICATED, O-PEE-CHEE, PACIFIC, PANINI, PARKHURST, PARKSIDE, PHILADELPHIA GUM, PINNACLE, PLAYOFF, POST, PRESS PASS, PROCARDS, PRO SET, R.E.L., RAZOR, RED MAN, REGINA, RINGSIDE, RITTENHOUSE, ROOX LIMITED CORP., SAGE, SCANLENS, SCORE, SELECT, SEREAL, SHIRRIFF COINS, SIGNATURE ROOKIES, SKYBOX, SP AUTHENTIC, SPORTFLICS, SPORT KINGS, SPORTSCASTER, SPORTS ILLUSTRATED, SPX, STADIUM CLUB, STAR PICS, SUPERIOR PIX, SUPERIOR ROOKIES, TCMA, TEAM ISSUE, THE SCORE BOARD, INC, THE STAR CO., TOPPS, TOPPS SUPREME, TRISTAR, TY, ULTIMATE GUARD, ULTIMATE TRADING CARD CO., ULTRA PRO, UPPER DECK, VACHON, WHEATIES, WHEELS, WILD CARD, WILLS, WIZARDS OF THE COAST, WOW, UNBRANDED,NOT SPECIFIED,5finity,7th inning sketch,aaa sports memorabilia,ace authentic,action packed,aeg,allens,american caramel,apba,artbox,atlantic,aw sports, inc.,bandaibaseball magazine,bazooka, be a player, bench warmer, bleachers, bowman, bushiroad, calbee, callahan hall of fame, cardz distribution, champion cards, choice marketing, inc, chris martin enterprises, inc., classic games, inc, classic marketing, inc., cmc, coles, collector's edge, college classics, inc., collegiate collection, comic images, courtside, cracker jack, cryptozoic, dan dee, diamond kings, diamond stars, disney, donruss, donruss, playoff, double play, eclipse cards, enor, exhibits, fantasy flight games, félix potin, fleer, fleer/skybox international, futera, goal line, goal line art, goudey, grandstand, hasbro, hi-pro marketing, inc., homogenized bond, impel, imperial tobacco canada, in the game, jogo, inc., just memorabilia, just minors, kahn's, kellogg's, kenner, konami, leaf, marvel, merlin, milk duds, mobil, monty gum, msa, national chicle, navarrete, nba properties, netpro, nfl players association, nfl properties, nintendo, nsi marketing limited, nu-cards, onyx authenticated, o-pee-chee, pacific, panini, parkhurst, parkside, philadelphia gum, pinnacle, playoff, post, press pass, procards, pro set, r.e.l., razor, red man, regina, ringside, rittenhouse, roox limited corp., sage, scanlens, score, select, sereal, shirriff coins, signature rookies, skybox, sp authentic, sportflics, sport kings, sportscaster, sports illustrated, spx, stadium club, star pics, superior pix, superior rookies, tcma, team issue, the score board, inc, the star co., topps, topps supreme, tristar, ty, ultimate guard, ultimate trading card co., ultra pro, upper deck, vachon, wheaties, wheels, wild card, wills, wizards of the coast, wow, unbranded,not specified,5FINITY,7th Inning Sketch,AAA Sports Memorabilia,Ace Authentic,Action Packed,AEG,Allens,American Caramel,APBA,Artbox,Atlantic,AW Sports, Inc.,Bandaibaseball Magazine,Bazooka, Be A Player, Bench Warmer, Bleachers, Bowman, Bushiroad, Calbee, Callahan HALL OF FAME, CARDZ Distribution, Champion Cards, Choice Marketing, Inc, Chris Martin Enterprises, Inc., Classic Games, Inc, Classic Marketing, Inc., CMC, Coles, Collector's Edge, College Classics, Inc., Collegiate Collection, Comic Images, Courtside, Cracker Jack, Cryptozoic, Dan Dee, Diamond Kings, Diamond Stars, Disney, Donruss, Donruss, Playoff, Double Play, Eclipse Cards, Enor, Exhibits, Fantasy Flight Games, Félix Potin, Fleer, Fleer/Skybox International, Futera, Goal Line, Goal Line Art, Goudey, Grandstand, Hasbro, Hi-Pro Marketing, Inc., Homogenized Bond, Impel, Imperial Tobacco Canada, In The Game, JOGO, Inc., Just Memorabilia, Just Minors, Kahn's, Kellogg's, Kenner, Konami, Leaf, Marvel, Merlin, Milk Duds, Mobil, Monty Gum, MSA, National Chicle, Navarrete, NBA Properties, Netpro, NFL Players Association, NFL Properties, Nintendo, NSI Marketing Limited, Nu-Cards, Onyx Authenticated, O-Pee-Chee, Pacific, Panini, Parkhurst, Parkside, Philadelphia Gum, Pinnacle, Playoff, Post, Press Pass, Procards, Pro Set, R.E.L., Razor, Red Man, Regina, Ringside, Rittenhouse, Roox Limited Corp., SAGE, Scanlens, SCORE, Select, Sereal, Shirriff Coins, Signature Rookies, Skybox, Sp Authentic, Sportflics, Sport Kings, Sportscaster, Sports Illustrated, SPX, Stadium Club, Star Pics, Superior Pix, Superior Rookies, TCMA, Team Issue, The Score Board, Inc, The Star Co., Topps, Topps Supreme, Tristar, Ty, Ultimate Guard, Ultimate Trading Card Co., Ultra PRO, Upper Deck, Vachon, Wheaties, Wheels, Wild Card, Wills, Wizards Of The Coast, WOW, Unbranded,Not Specified"

In [68]:
# Convert the brands string to a list
brand_list=string.split(',')

In [69]:
# strip forward whitespace 
brand_list=[x.strip() for x in brand_list]

In [70]:

len(brand_list)

612

There are 612 brand names we can look fro in title column.

In [71]:
# inspect brands list
brand_list

['5FINITY',
 '7th Inning Sketch',
 'AAA Sports Memorabilia',
 'Ace Authentic',
 'Action Packed',
 'AEG',
 'Allens',
 'American Caramel',
 'APBA',
 'ArtBox',
 'Atlantic',
 'AW Sports',
 'Inc.',
 'BandaiBaseball Magazine',
 'Bazooka',
 'Be A Player',
 'Bench Warmer',
 'Bleachers',
 'Bowman',
 'Bushiroad',
 'Calbee',
 'Callahan HALL OF FAME',
 'CARDZ Distribution',
 'Champion Cards',
 'Choice Marketing',
 'Inc',
 'Chris Martin Enterprises',
 'Inc.',
 'Classic Games',
 'Inc',
 'Classic Marketing',
 'Inc.',
 'CMC',
 'Coles',
 "Collector's Edge",
 'College Classics',
 'Inc.',
 'Collegiate Collection',
 'Comic Images',
 'Courtside',
 'Cracker Jack',
 'Cryptozoic',
 'Dan Dee',
 'Diamond Kings',
 'Diamond Stars',
 'Disney',
 'Donruss',
 'Donruss',
 'Playoff',
 'Double Play',
 'Eclipse Cards',
 'Enor',
 'Exhibits',
 'Fantasy Flight Games',
 'Félix Potin',
 'Fleer',
 'Fleer/SkyBox International',
 'Futera',
 'Goal Line',
 'Goal Line Art',
 'Goudey',
 'Grandstand',
 'Hasbro',
 'Hi-Pro Marketing',


Now that we have a brand list ready, let’s search for the brands in the text of the title column and extract them to a new column.

In [72]:
# grab brand from title column if it is in brand_list
df['card_brand'] = df['title'].str.extract('({})'.format('|'.join(brand_list)), expand=True)

In [73]:
# show Nan values from card_brand column
df[df['card_brand'].isnull()][['card_brand','url','title']].head(30)


,card_brand,url,title
1,NaN,https://www.ebay.com/itm/195083288951,SUPER RARE ( ELITE STATUS ) 2000 TOM BRADY ROOKIE DONURSS S/N 05/10 PSA 10 POP 1
2,NaN,https://www.ebay.com/itm/285024859926,2003 LEBRON JAMES RC UD ROOKIE EXCLUSIVES AUTO GEM MINT Rare Highest Graded
4,NaN,https://www.ebay.com/itm/204074772841,RARE 2007 Tom Brady Autograph Plaque 16-0 #Undefeated #GreatnessCode #RookieCard
5,NaN,https://www.ebay.com/itm/265932738980,1947 Jackie Robinson Rookie RC Brooklyn Dodgers 1st MLB Appearance PSA 9 Mint
6,NaN,https://www.ebay.com/itm/383932760088,2015-16 Exquisite Connor McDavid 1/1 RPA NHL Shield Logo Patch BGS 9.5 10 AUTO
7,NaN,https://www.ebay.com/itm/144603360662,Lebron James 2003 SP Signature Edition RC Inscription 'The KING' Auto /25 BGS10
9,NaN,https://www.ebay.com/itm/284482688684,2003-04 UD Rookie Exclusives AUTOGRAPH Michael Jordan A60 BGS PRISTINE 10 POP 1!
12,NaN,https://www.ebay.com/itm/175281488184,2009 National Treasures Stephen Curry RC Jersey Auto /99 RPA BGS 9
17,NaN,https://www.ebay.com/itm/164779971947,2003 Ultimate LeBron James ROOKIE AUTO /250 #127 BGS PRISTINE 10! Holy Grail!!
18,NaN,https://www.ebay.com/itm/264275394203,1997 Browns Boxing #51 Floyd Mayweather Jr Rookie RC BGS 10 PRISTINE


In [74]:
# show number of Nan values from card_brand column
df.card_brand.isnull().sum()

13678

Seems like most of the missing brands names are due to non popular brands or sets of cards instead of single listings. Let's drop them.

In [75]:
# drop rows with Nan values in card_brand column
df.dropna(subset=['card_brand'], inplace=True)

# check
df.card_brand.isnull().sum()

0

In [76]:
# Check found brand names
df.card_brand.value_counts().head(20)

Panini        11214
Topps          9956
Bowman         5228
TOPPS          2528
PANINI         2125
Fleer          1572
Donruss        1469
Upper Deck     1373
Select         1066
BOWMAN          902
Leaf            652
UPPER DECK      432
FLEER           425
Skybox          292
Ty              272
Goudey          270
DONRUSS         269
panini          243
SELECT          233
Playoff         233
Name: card_brand, dtype: int64

There is a repetition of brand categories due to the case sensitivity in letters. We can create a dictionary with possible case combinations and map to concatenate such repetitive names.

In [77]:
# map values in card_brand to a custom dictionary
brand_dict = {'PANINI':'Panini','panini':'Panini',
              'TOPPS':'Topps','topps':'Topps',
              'BOWMAN':'Bowman','bowman':'Bowman',
              'UD':'Upper Deck','ud':'Upper Deck','UPPER DECK':'Upper Deck','upper deck':'Upper Deck',
              'DONRUSS':'Donruss','donruss':'Donruss',
              'FLEER':'Fleer','fleer':'Fleer',
              'SELECT':'Select','select':'Select',
              'LEAF':'Leaf','leaf':'Leaf',
              'SKYBOX':'Skybox','skybox':'Skybox',
              'SP':'SportsPicks','sp':'SportsPicks','SPORTSPICKS':'SportsPicks','sportspicks':'SportsPicks',
              'TRISTAR':'Tristar','tristar':'Tristar',
              'TY':'ty','ty':'Ty',
              'PACIFIC':'Pacific','pacific':'Pacific',
              'SP AUTHENTIC':'SP Authentic','sp authentic':'SP Authentic',
              'Spx':'SPX','spx':'SPX',
              'PARKHURST':'Parkhurst','parkhurst':'Parkhurst',
              'GOUDEY':'Goudey','goudey':'Goudey',
              'PLAYOFF':'Playoff','playoff':'Playoff',
              'O-PEE-CHEE':'O-Pee-Chee','o-pee-chee':'O-Pee-Chee',
              'STADIUM CLUB':'Stadium Club','stadium club':'Stadium Club',
              'PRESS PASS':'Press Pass','press pass':'Press',
              'BAZOOKA':'Bazooka','bazooka':'Bazooka',
              'NETPRO':'NetPro','netpro':'NetPro',
              'POST':'Post','post':'Post',
              'SCORE':'Score','score':'Score',
              'AMERICAN CARAMEL':'American Caramel','american caramel':'American Caramel',
              'EXHIBITS':'Exhibits','exhibits':'Exhibits',
              'MERLIN':'Merlin','merlin':'Merlin',
              'INCH':'Inch','inch':'Inch',
              'SIGNATURE ROOKIES':'Signature Rookies','signature rookies':'Signature Rookies',
              'PRO SET':'Pro Set','pro set':'Pro Set',
              'ENOR': 'Enor','enor':'Enor'}

# replace values in card_brand column with values in brand_dict if they match, else leave as is
df['card_brand'] = df['card_brand'].map(brand_dict).fillna(df['card_brand'])
              



In [78]:
# check
df.card_brand.value_counts().head(20)

Panini          13582
Topps           12713
Bowman           6224
Fleer            2045
Upper Deck       1834
Donruss          1764
Select           1352
Leaf              853
Skybox            403
Ty                401
O-Pee-Chee        382
Goudey            306
Playoff           299
ty                186
Stadium Club      180
Pacific           158
Parkhurst         141
SP Authentic      109
SPX                95
Press Pass         81
Name: card_brand, dtype: int64

### Feature Engineering `card_singed` column

Whether a card is autographed or not can be a valuable feature for our analysis. We can use the title column to create a new column that will be a string "Yes" if the card is autographed and "No" if it is not.
To do so, we must first look for keywords in the text indicating that a card is signed. We can do this by creating a possible list of keyword combinations to look for.


In [80]:
# List of possible keywords to look for in title column
singed = ['Signed','signed','Autographed','autographed','Autograph','autograph','Autographed Card','autographed card','Autographed Card','autographed card']


In [81]:
# grab keyword from title column if it is in singed list and replace other with  'No'
df['card_singed'] = df['title'].str.extract('({})'.format('|'.join(singed)), expand=True).replace(np.nan, 'No')

# check
df.card_singed.value_counts()

No             42002
Autograph       1645
Signed           439
Autographed      133
autograph         55
signed            27
autographed       13
Name: card_singed, dtype: int64

All the other values aprt from 'No' are the same, so we can replace them with 'Yes'.

In [82]:
# if value from card_singed column is in singed list replace with 'Yes'
df['card_singed'] = df['card_singed'].replace(singed, 'Yes')

#check
df.card_singed.value_counts()

No     42002
Yes     2312
Name: card_singed, dtype: int64

---
># Conclustion

We have cleaned the dataset and created new meaningful features. Let's inspect and export the cleaned version that ready to be analyzed

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44314 entries, 0 to 60472
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         44314 non-null  float64
 1   title         44314 non-null  object 
 2   url           44314 non-null  object 
 3   whyToBuy/0    44314 non-null  object 
 4   item_country  44314 non-null  object 
 5   item_state    44314 non-null  object 
 6   item_city     44314 non-null  object 
 7   card_year     44314 non-null  object 
 8   card_brand    44314 non-null  object 
 9   card_singed   44314 non-null  object 
dtypes: float64(1), object(9)
memory usage: 3.7+ MB


In [84]:
# reset index
df.reset_index(drop=True, inplace=True)

# check
df.tail()

,price,title,url,whyToBuy/0,item_country,item_state,item_city,card_year,card_brand,card_singed
44309,237.83,Wander Franco 2019 1st Bowman Chrome Mega Box Refractor RC Rays BGS 9.5 #BCP100,https://www.ebay.com/itm/334660820379,Ships from United States,United States,New York,Oswego,2019,Bowman,No
44310,150.00,PSA9 MINT 2013 Antti Raanta EXCLUSIVES Upper Deck Young Guns ROOKIE,https://www.ebay.com/itm/266035221956,Ships from United States,United States,Arizona,Phoenix,2013,Upper Deck,No
44311,325.00,1955 BOWMAN MICKEY MANTLE #202 PSA 1,https://www.ebay.com/itm/266035228289,Ships from United States,United States,Wisconsin,Oshkosh,1955,Bowman,No
44312,124.99,2019-20 Panini Mosaic Lebron James Silver Mosaic Prizm PSA Gem Mint 10 #8 Lakers,https://www.ebay.com/itm/204182013716,Ships from United States,United States,Indiana,Evansville,2019,Panini,No
44313,295.00,2015-16 PANINI PRIZM DEVIN BOOKER PSA 10 ROOKIE RC #308 PHOENIX SUNS,https://www.ebay.com/itm/404045492577,Ships from United States,United States,Illinois,Frankfort,2015,Panini,No


In [85]:
df.shape

(44314, 10)

Our cleaned and preprocessed dataset is ready to be used for analysis with 44,314 rows and 10 columns of valuable data on recent sports trading card listings on eBay USA.

Let's export the cleaned version.


In [86]:
# save to csv
df.to_csv('psa data/ebay_cards_cleaned.csv', index=False)

In [87]:
# Sanity Check
# load cleaned data and check
df = pd.read_csv('psa data/ebay_cards_cleaned.csv')

# check
df.head()

,price,title,url,whyToBuy/0,item_country,item_state,item_city,card_year,card_brand,card_singed
0,12630069.0,1986 86 Fleer Michael Jordan Rookie Sticker PSA 8(ST) # 06300690 CAREER HIGH PTS,https://www.ebay.com/itm/324272400062,Ships from United States,United States,Michigan,Dearborn,1986,Fleer,No
1,4500000.0,2003 TOM BRADY LEAF LIMITED # 12/12 GAME-WORN-2002-PRO-BOWL-JERSEY-AUTO-BGS 9/10,https://www.ebay.com/itm/195509543378,Ships from United States,United States,Michigan,Livonia,2003,Leaf,No
2,3200000.0,RARE ( PLAYERS HONORS )(#/10) 2000 TOM BRADY ROOKIE ABSOLUTE PLAYOFF PSA 9 POP 1,https://www.ebay.com/itm/193723287418,Ships from United States,United States,Michigan,Livonia,2000,Playoff,No
3,2000000.0,2018-19 PANINI CONTENDERS LUKA DONCIC PREMIUM GOLD RC AUTO BGS 9.5/10 POP 1,https://www.ebay.com/itm/165123068721,Ships from United States,United States,California,Rancho Cucamonga,2018,Panini,No
4,1500000.0,2020 Topps Chrome F1 Lewis Hamilton Silver Refractor Auto #F1A-LH (44/58) PSA 10,https://www.ebay.com/itm/194656052157,Free shipping and returns,United States,Virginia,Sterling,2020,Topps,No


---